# 라이브러리

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#########################################################################
import matplotlib.font_manager as fm
import matplotlib as mpl
#########################################################################
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
#########################################################################
from tqdm import tqdm

# 1. 한글 폰트 경로 설정 (Windows 기본: 맑은 고딕)
font_path = "C:/Windows/Fonts/malgun.ttf"
font_name = fm.FontProperties(fname=font_path).get_name()

# 2. matplotlib에 폰트 적용
mpl.rc("font", family=font_name)

# 3. 마이너스 기호 깨짐 방지
mpl.rcParams["axes.unicode_minus"] = False

In [ ]:
jemu_df = pd.read_csv('../../data/재무정보_final.csv', encoding='cp949')
jemu_df.tail(1)

# 전체 데이터 eda

## DF 형태 파악

In [ ]:
# 고유 기업명만
jemu_df["기업명"].unique()

In [ ]:
# 특정 기업에 대한 행 보기
jemu_df[jemu_df["기업명"] == "휴비오 주식회사"]

### 컬럼명들

In [ ]:
jemu_df.columns

In [ ]:
jemu_df.info()

### 기업별 행 개수
- 모든 값이 다 6이 나와야 정상 -> ok

In [ ]:
pd.set_option("display.max_rows", None) # 모든 행 보기
jemu_df.groupby("기업명").size().sort_values(ascending=False)

### 기업별 결측치 비율
- 1에 가까울수록 거의 다 결측

In [ ]:
company_missing_ratio = (
    jemu_df
    .groupby("기업명")
    .apply(lambda x: x.isna().mean())
)

company_missing_ratio.head()

## 변수 기준으로 기업별 분포 보기
- (우선) 변수 기준 : 매출액
-> 이렇게 하면 빈 값 확인 가능

In [ ]:
jemu_df.columns = jemu_df.columns.str.strip()

In [ ]:
jemu_df.columns.tolist()

In [ ]:
pd.set_option("display.max_rows", None) # 모든 행 보기
jemu_df.groupby("기업명")['매출액'].count().sort_values()   # 기업별 매출액에 값 들어있는 개수

### 매출액

In [ ]:
sales_count = (
    jemu_df
    .groupby("기업명")['매출액']
    .count()
)

count_distribution = sales_count.value_counts().sort_index()

plt.figure()
bars =plt.bar(
    count_distribution.index,
    count_distribution.values
)
plt.xticks(count_distribution.index)
plt.xlabel("기업별 매출액 값 개수 (연도 수)")
plt.ylabel("기업 수")
plt.title("매출액 데이터 보유 개수별 기업 분포")

for bar in bars:
    height = bar.get_height()
    plt.text(
        bar.get_x() + bar.get_width() / 2,
        height,
        int(height),
        ha="center",
        va="bottom"
    )

plt.show()

### 연구개발비

In [ ]:
sales_count = (
    jemu_df
    .groupby("기업명")["연구개발비"]
    .count()
)

count_distribution = sales_count.value_counts().sort_index()

plt.figure()
bars =plt.bar(
    count_distribution.index,
    count_distribution.values
)
plt.xticks(count_distribution.index)
plt.xlabel("기업별 연구개발비 값 개수 (연도 수)")
plt.ylabel("기업 수")
plt.title("연구개발비 데이터 보유 개수별 기업 분포")

for bar in bars:
    height = bar.get_height()
    plt.text(
        bar.get_x() + bar.get_width() / 2,
        height,
        int(height),
        ha="center",
        va="bottom"
    )

plt.show()

In [ ]:
pd.set_option("display.max_rows", None) # 모든 행 보기

ales_count = (
    jemu_df
    .groupby("기업명")["매출액"]
    .count()
)

zero_sales_companies = sales_count[sales_count == 6]

zero_sales_companies.index.tolist()

# KNN-test
        1) 같은 연도 + 같은 산업(sectors/field) 후보 추출
        2) 이웃 기준 컬럼으로 KNN 이웃 K개 선택
        3) 그 이웃을 공통으로 사용
        4) 8개 재무 컬럼을 성격별 규칙으로 대체
        5) KNN 실패 시 Cold deck fallback

- 설정값

In [ ]:
# 하드 필터 컬럼
INDUSTRY_COL = "sectors"   # field 대신 sectors 사용

# 🔥 [FIX] 이웃 기준 컬럼 (고정, 숫자만)
NEIGHBOR_COLS = ["자산", "자본", "종업원_합계"]

# 결측치 대체 대상 컬럼
FLOW_COLS   = ["매출액", "영업이익", "당기순이익"]
STOCK_COLS  = ["부채"]
INVEST_COLS = ["연구개발비", "CAPEX"]

TARGET_COLS = FLOW_COLS + STOCK_COLS + INVEST_COLS


- 전처리

In [ ]:
# =========================
# 0. 컬럼명 정규화 (필수)
# =========================
# 🔥 [FIX] NBSP / 공백 문제 해결
jemu_df.columns = (
    jemu_df.columns
    .str.replace("\xa0", " ", regex=False)
    .str.strip()
)

# =========================
# 🔥 [NEW] KNN / Cold deck 전 필수 전처리
# =========================
# 🔥 [FIX] 이웃 기준 컬럼 숫자 변환
for col in NEIGHBOR_COLS:
    jemu_df[col] = pd.to_numeric(jemu_df[col], errors="coerce")

# 연도 숫자 변환
jemu_df["연도"] = pd.to_numeric(jemu_df["연도"], errors="coerce")

# 🔥 [FIX] 대체 대상 컬럼 숫자 변환
for col in TARGET_COLS:
    jemu_df[col] = pd.to_numeric(jemu_df[col], errors="coerce")

# 🔥 [NEW] _filled 컬럼
for col in TARGET_COLS:
    if f"{col}_filled" not in jemu_df.columns:
        jemu_df[f"{col}_filled"] = jemu_df[col]

# 🔥 [NEW] 컬럼별 imputation method 로그
for col in TARGET_COLS:
    if f"{col}_imputation_method" not in jemu_df.columns:
        jemu_df[f"{col}_imputation_method"] = pd.NA

# 🔥 [FIX] 원래 값은 절대 KNN / COLD_DECK으로 찍히지 않게
for col in TARGET_COLS:
    jemu_df.loc[
        jemu_df[col].notna(),
        f"{col}_imputation_method"
    ] = "ORIGINAL"

- 기업–연도 단위 KNN 이웃 찾기 함수 (완화 버전)

In [ ]:
def find_knn_neighbors_by_year(df, company, year, k=3):

    target = df[
        (df["기업명"] == company) &
        (df["연도"] == year)
    ]
    if target.empty:
        return None

    industry = target[INDUSTRY_COL].values[0]

    # 🔥 [FIX] 산업 정보 없으면 KNN 불가
    if pd.isna(industry):
        return None

    # 🔥 [FIX] 이웃 기준 컬럼 중 2개 이상 필요
    if target[NEIGHBOR_COLS].notna().sum(axis=1).values[0] < 2:
        return None

    # 하드 필터
    candidates = df[
        (df["연도"] == year) &
        (df[INDUSTRY_COL] == industry) &
        (df["기업명"] != company)
    ].copy()

    # 후보도 구조 정보 2개 이상 필요
    candidates = candidates[
        candidates[NEIGHBOR_COLS].notna().sum(axis=1) >= 2
    ]

    if len(candidates) < k:
        return None

    # 거리 계산용 데이터 구성
    X_cand = candidates[NEIGHBOR_COLS].fillna(candidates[NEIGHBOR_COLS].mean())
    X_tgt  = target[NEIGHBOR_COLS].fillna(candidates[NEIGHBOR_COLS].mean())

    scaler = StandardScaler()
    X_all = pd.concat([X_cand, X_tgt])
    X_scaled = scaler.fit_transform(X_all)

    knn = NearestNeighbors(n_neighbors=k)
    knn.fit(X_scaled[:-1])
    _, idx = knn.kneighbors(X_scaled[-1].reshape(1, -1))

    return candidates.iloc[idx[0]]


- KNN 이웃 기반 결측 대체

In [ ]:
def impute_from_neighbors(neighbors):
    result = {}

    for col in FLOW_COLS:
        v = neighbors[col].dropna()
        if len(v) > 0:
            result[col] = v.median()

    for col in STOCK_COLS:
        v = neighbors[col].dropna()
        if len(v) > 0:
            result[col] = v.median()

    for col in INVEST_COLS:
        v = neighbors[col].dropna()
        result[col] = v.median() if len(v) > 0 else 0

    return result

- Cold deck (KNN 불가 시 대체)

In [ ]:
def cold_deck_impute(df, company, year):

    target = df[
        (df["기업명"] == company) &
        (df["연도"] == year)
    ]
    if target.empty:
        return {}

    industry = target[INDUSTRY_COL].values[0]
    result = {}

    for col in TARGET_COLS:

        if not pd.isna(target[col].values[0]):
            continue

        vals = df[
            (df["연도"] == year) &
            (df[INDUSTRY_COL] == industry)
        ][col].dropna()

        result[col] = vals.median() if len(vals) > 0 else df[col].median()

    return result

- 전체 실행 루프 (KNN + Cold deck 통합)

In [ ]:
for company in tqdm(
    jemu_df["기업명"].unique(),
    desc="기업 단위 결측치 보완"
):

    company_df = jemu_df[jemu_df["기업명"] == company]

    missing_years = company_df[
        company_df[TARGET_COLS].isna().any(axis=1)
    ]["연도"].unique()

    for year in missing_years:

        neighbors = find_knn_neighbors_by_year(
            df=jemu_df,
            company=company,
            year=year,
            k=3
        )

        if neighbors is not None:
            imputed = impute_from_neighbors(neighbors)
            method = "KNN"
        else:
            imputed = cold_deck_impute(jemu_df, company, year)
            method = "COLD_DECK"

        for col, value in imputed.items():

            mask = (
                (jemu_df["기업명"] == company) &
                (jemu_df["연도"] == year) &
                (jemu_df[col].isna())
            )

            # 값 채우기
            jemu_df.loc[mask, f"{col}_filled"] = value

            # 🔥 [FIX] 채워진 셀에만 method 기록
            jemu_df.loc[mask, f"{col}_imputation_method"] = method

In [ ]:
for col in TARGET_COLS:
    print(col)
    print(jemu_df[f"{col}_imputation_method"].value_counts())
    print("-" * 40)

In [ ]:
for col in TARGET_COLS:
    filled_col = f"{col}_filled"

    filled_count = (
        jemu_df[col].isna() &
        jemu_df[filled_col].notna()
    ).sum()

    print(f"{col}: 새로 채워진 개수 = {filled_count}")

In [ ]:
jemu_df["imputation_method"].value_counts(dropna=False)


In [ ]:
jemu_df.head()

In [ ]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
import warnings
warnings.filterwarnings('ignore')

# ========================================
# 공통 설정
# ========================================
FINANCIAL_COLS = [
    '매출액', '영업이익', '당기순이익',
    '자산', '부채', '자본',
    '연구개발비', 'CAPEX'
]

INDUSTRY_COL = 'sectors'
YEAR_COL = '연도'
COMPANY_COL = '기업명'


# ========================================
# 🔴 [수정 1] 재무 컬럼 숫자형 강제 변환
# ========================================
def ensure_numeric_financial_cols(df):
    df = df.copy()
    df[FINANCIAL_COLS] = df[FINANCIAL_COLS].apply(
        pd.to_numeric, errors='coerce'
    )
    return df


# ========================================
# 1단계: 기업 프로필 생성
# ========================================
def create_company_profile(df):
    company_profile = (
        df.groupby(COMPANY_COL)
        .agg({
            '매출액': 'median',
            '자산': 'median',
            INDUSTRY_COL: 'first'
        })
        .reset_index()
    )

    company_profile['규모'] = pd.cut(
        company_profile['매출액'],
        bins=[0, 100, 500, 1000, np.inf],
        labels=['소형', '중소형', '중형', '대형']
    )

    company_profile['규모'] = company_profile['규모'].cat.add_categories(['Unknown'])
    company_profile.loc[company_profile['매출액'].isnull(), '규모'] = 'Unknown'

    return company_profile


# ========================================
# 2단계: 유사 기업 중앙값 대체
# ========================================
def impute_by_similar_companies(df, company_profile):
    df_imputed = df.copy()
    missing_rows = df[df[FINANCIAL_COLS].isnull().any(axis=1)]

    for idx, row in missing_rows.iterrows():
        company = row[COMPANY_COL]
        year = row[YEAR_COL]

        profile_row = company_profile[company_profile[COMPANY_COL] == company]
        if profile_row.empty:
            continue
        profile = profile_row.iloc[0]

        similar_companies = []

        if profile['규모'] != 'Unknown':
            similar_companies = company_profile[
                (company_profile[INDUSTRY_COL] == profile[INDUSTRY_COL]) &
                (company_profile['규모'] == profile['규모']) &
                (company_profile[COMPANY_COL] != company)
            ][COMPANY_COL].tolist()

        if len(similar_companies) < 3:
            similar_companies = company_profile[
                (company_profile[INDUSTRY_COL] == profile[INDUSTRY_COL]) &
                (company_profile[COMPANY_COL] != company)
            ][COMPANY_COL].tolist()

        if len(similar_companies) < 3 and profile['규모'] != 'Unknown':
            similar_companies = company_profile[
                (company_profile['규모'] == profile['규모']) &
                (company_profile[COMPANY_COL] != company)
            ][COMPANY_COL].tolist()

        similar_data = df[
            (df[COMPANY_COL].isin(similar_companies)) &
            (df[YEAR_COL] == year)
        ]

        for col in FINANCIAL_COLS:
            if pd.isnull(row[col]) and similar_data[col].dropna().shape[0] >= 3:
                df_imputed.at[idx, col] = similar_data[col].median()

    return df_imputed


# ========================================
# 3단계: KNN Imputation (산업 + 연도)
# ========================================
def knn_impute_by_industry_year(df, n_neighbors=5):
    df_imputed = df.copy()

    for (industry, year), group in df.groupby([INDUSTRY_COL, YEAR_COL]):
        if len(group) <= n_neighbors:
            continue

        imputer = KNNImputer(
            n_neighbors=min(n_neighbors, len(group) - 1),
            weights='distance'
        )

        imputed = imputer.fit_transform(group[FINANCIAL_COLS])
        df_imputed.loc[group.index, FINANCIAL_COLS] = imputed

    return df_imputed


# ========================================
# 4단계: 통합 실행
# ========================================
def hybrid_imputation(df):
    df_result = ensure_numeric_financial_cols(df)  # 🔴 여기서 먼저 변환

    print("=" * 50)
    print("결측치 처리 시작")
    print("=" * 50)
    print(f"초기 결측치: {df_result[FINANCIAL_COLS].isnull().sum().sum()}개\n")

    company_profile = create_company_profile(df_result)
    print(f"기업 프로필 생성 완료: {len(company_profile)}개 기업")

    print("\n[Step 1] 유사 기업 중앙값 대체 중...")
    df_result = impute_by_similar_companies(df_result, company_profile)
    remaining = df_result[FINANCIAL_COLS].isnull().sum().sum()
    print(f"✓ 완료 | 남은 결측치: {remaining}개")

    if remaining > 0:
        print("\n[Step 2] KNN 대체 중 (산업+연도 기준)...")
        df_result = knn_impute_by_industry_year(df_result)
        remaining = df_result[FINANCIAL_COLS].isnull().sum().sum()
        print(f"✓ 완료 | 남은 결측치: {remaining}개")

    print("\n결측치 처리 완료")
    print("=" * 50)

    return df_result, company_profile


# ========================================
# 5단계: 실행
# ========================================
df_final, company_profile = hybrid_imputation(jemu_df)

print("\n[변수별 남은 결측치]")
print(df_final[FINANCIAL_COLS].isnull().sum())


In [ ]:
df_final.to_csv("../../output/jemu_df_imputed.csv", index=False, encoding="cp949")

In [ ]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
import warnings
warnings.filterwarnings('ignore')

# ========================================
# 공통 설정
# ========================================
FINANCIAL_COLS = [
    '매출액', '영업이익', '당기순이익',
    '자산', '부채', '자본',
    '연구개발비', 'CAPEX'
]

INDUSTRY_COL = 'field'
YEAR_COL = '연도'
COMPANY_COL = '기업명'


# ========================================
# 🔴 [수정 1] 재무 컬럼 숫자형 강제 변환
# ========================================
def ensure_numeric_financial_cols(df):
    df = df.copy()
    df[FINANCIAL_COLS] = df[FINANCIAL_COLS].apply(
        pd.to_numeric, errors='coerce'
    )
    return df


# ========================================
# 1단계: 기업 프로필 생성
# ========================================
def create_company_profile(df):
    company_profile = (
        df.groupby(COMPANY_COL)
        .agg({
            '매출액': 'median',
            '자산': 'median',
            INDUSTRY_COL: 'first'
        })
        .reset_index()
    )

    company_profile['규모'] = pd.cut(
        company_profile['매출액'],
        bins=[0, 100, 500, 1000, np.inf],
        labels=['소형', '중소형', '중형', '대형']
    )

    company_profile['규모'] = company_profile['규모'].cat.add_categories(['Unknown'])
    company_profile.loc[company_profile['매출액'].isnull(), '규모'] = 'Unknown'

    return company_profile


# ========================================
# 2단계: 유사 기업 중앙값 대체
# ========================================
def impute_by_similar_companies(df, company_profile):
    df_imputed = df.copy()
    missing_rows = df[df[FINANCIAL_COLS].isnull().any(axis=1)]

    for idx, row in missing_rows.iterrows():
        company = row[COMPANY_COL]
        year = row[YEAR_COL]

        profile_row = company_profile[company_profile[COMPANY_COL] == company]
        if profile_row.empty:
            continue
        profile = profile_row.iloc[0]

        similar_companies = []

        if profile['규모'] != 'Unknown':
            similar_companies = company_profile[
                (company_profile[INDUSTRY_COL] == profile[INDUSTRY_COL]) &
                (company_profile['규모'] == profile['규모']) &
                (company_profile[COMPANY_COL] != company)
            ][COMPANY_COL].tolist()

        if len(similar_companies) < 3:
            similar_companies = company_profile[
                (company_profile[INDUSTRY_COL] == profile[INDUSTRY_COL]) &
                (company_profile[COMPANY_COL] != company)
            ][COMPANY_COL].tolist()

        if len(similar_companies) < 3 and profile['규모'] != 'Unknown':
            similar_companies = company_profile[
                (company_profile['규모'] == profile['규모']) &
                (company_profile[COMPANY_COL] != company)
            ][COMPANY_COL].tolist()

        similar_data = df[
            (df[COMPANY_COL].isin(similar_companies)) &
            (df[YEAR_COL] == year)
        ]

        for col in FINANCIAL_COLS:
            if pd.isnull(row[col]) and similar_data[col].dropna().shape[0] >= 3:
                df_imputed.at[idx, col] = similar_data[col].median()

    return df_imputed


# ========================================
# 3단계: KNN Imputation (산업 + 연도)
# ========================================
def knn_impute_by_industry_year(df, n_neighbors=5):
    df_imputed = df.copy()

    for (industry, year), group in df.groupby([INDUSTRY_COL, YEAR_COL]):
        if len(group) <= n_neighbors:
            continue

        imputer = KNNImputer(
            n_neighbors=min(n_neighbors, len(group) - 1),
            weights='distance'
        )

        imputed = imputer.fit_transform(group[FINANCIAL_COLS])
        df_imputed.loc[group.index, FINANCIAL_COLS] = imputed

    return df_imputed


# ========================================
# 4단계: 통합 실행
# ========================================
def hybrid_imputation(df):
    df_result = ensure_numeric_financial_cols(df)  # 🔴 여기서 먼저 변환

    print("=" * 50)
    print("결측치 처리 시작")
    print("=" * 50)
    print(f"초기 결측치: {df_result[FINANCIAL_COLS].isnull().sum().sum()}개\n")

    company_profile = create_company_profile(df_result)
    print(f"기업 프로필 생성 완료: {len(company_profile)}개 기업")

    print("\n[Step 1] 유사 기업 중앙값 대체 중...")
    df_result = impute_by_similar_companies(df_result, company_profile)
    remaining = df_result[FINANCIAL_COLS].isnull().sum().sum()
    print(f"✓ 완료 | 남은 결측치: {remaining}개")

    if remaining > 0:
        print("\n[Step 2] KNN 대체 중 (산업+연도 기준)...")
        df_result = knn_impute_by_industry_year(df_result)
        remaining = df_result[FINANCIAL_COLS].isnull().sum().sum()
        print(f"✓ 완료 | 남은 결측치: {remaining}개")

    print("\n결측치 처리 완료")
    print("=" * 50)

    return df_result, company_profile


# ========================================
# 5단계: 실행
# ========================================
df_final, company_profile = hybrid_imputation(jemu_df)

print("\n[변수별 남은 결측치]")
print(df_final[FINANCIAL_COLS].isnull().sum())


In [ ]:
df_final.to_csv("../../output/jemu_df_imputed_fields.csv", index=False, encoding="cp949")